In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

#import os


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)




==> Preparing data..
Files already downloaded and verified


In [2]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


net = resnet18()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)



lr = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)



Files already downloaded and verified
==> Building model..


In [3]:
def PGDPerturb(model, x, y):
    """Performs the projected gradient descent attack on a batch of images."""

    epsilon = 0.0314
    alpha = 0.00784
    num_steps = 7
    clamp=(0,1)

    x_adv = x.clone().detach().requires_grad_(True).to(x.device)
    x_adv =  x_adv + torch.zeros_like(x_adv).uniform_(-epsilon, epsilon)
    num_channels = x.shape[1]

    for i in range(num_steps):
        _x_adv = x_adv.clone().detach().requires_grad_(True)
        prediction = model(_x_adv)
        loss = criterion(prediction,y)
        loss.backward()

        with torch.no_grad():
            # Force the gradient step to be a fixed size in a certain norm
            gradients = _x_adv.grad.sign() * alpha
            x_adv += gradients
            x_adv = torch.max(torch.min(x_adv, x + epsilon), x - epsilon)
            x_adv = x_adv.clamp(*clamp)

    return x_adv.detach()


In [4]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        perturbed_data = PGDPerturb(net, inputs, targets)
        optimizer.zero_grad()
        outputs = net(perturbed_data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print('Train Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [5]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print('Test Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/resnet18.pth')
        best_acc = acc


In [ ]:
for epoch in range(0, 200):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
Train Loss: 2.570 | Acc: 15.648% (7824/50000)
Test Loss: 1.998 | Acc: 24.550% (2455/10000)
Saving..

Epoch: 1
Train Loss: 2.133 | Acc: 20.298% (10149/50000)
Test Loss: 1.861 | Acc: 30.770% (3077/10000)
Saving..

Epoch: 2
Train Loss: 2.063 | Acc: 22.906% (11453/50000)
Test Loss: 1.755 | Acc: 37.550% (3755/10000)
Saving..

Epoch: 3
Train Loss: 2.022 | Acc: 24.318% (12159/50000)
Test Loss: 1.710 | Acc: 37.940% (3794/10000)
Saving..

Epoch: 4


In [ ]:
#if resume:
    # Load checkpoint.
#    print('==> Resuming from checkpoint..')
#    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#    checkpoint = torch.load('./checkpoint/ckpt.pth')
#    net.load_state_dict(checkpoint['net'])
#    best_acc = checkpoint['acc']
#    start_epoch = checkpoint['epoch']

